In [14]:
### OLD CODE ###
import pyparsing as pp

class Proposition:
    pass

class PropVariable(Proposition):
    def __init__(self, v):
        assert (type(v) == type("") and len(v) == 1) 
        self.v = v
    def __str__(self):
        return(self.v)
    def eval(self, prop_eval):
        return prop_eval(self.v)
    def __eq__(self, p):
        if type(p) == PropVariable: return p.v == self.v
        return False

class Negation(Proposition):
    def __init__(self, beta):
        assert issubclass(type(beta),Proposition)
        self.neg = beta
    def __str__(self):
        return("(not " + str(self.neg) + ")")
    def eval(self, prop_eval):
        return(not self.neg.eval(prop_eval))
    def __eq__(self, p):
        if type(p) == Negation: return p.neg == self.neg
        return False

class Disjunction(Proposition):
    def __init__(self, beta, gamma):
        assert issubclass(type(beta),Proposition) \
            and issubclass(type(gamma),Proposition)
        self.left, self.right = beta, gamma
    def __str__(self):
        return("(" + str(self.left )+" or " + str(self.right) + ")")
    def eval(self, prop_eval):
        l, r = self.left.eval(prop_eval), self.right.eval(prop_eval)
        return( l or r )
    def __eq__(self, p): 
        if type(p) == Disjunction: 
            return p.left == self.left and p.right == self.right
        return False

class Conditional(Proposition):
    def __init__(self, beta, gamma):
        assert issubclass(type(beta),Proposition) \
            and issubclass(type(gamma),Proposition)
        self.ante, self.conseq = beta, gamma
    def __str__(self):
        return("(if " + str(self.ante) + " then " + str(self.conseq) + ")")
    def eval(self, prop_eval):
        ante, conseq = self.ante.eval(prop_eval), self.conseq.eval(prop_eval)
        return (not ante) or conseq
    def __eq__(self, p) -> bool:
        if type(p) == Conditional:
            return p.ante==self.ante and p.conseq==self.conseq
        return False

def parseTree(p):
    if type(p) == type(""):
        return PropVariable(p)
    if len(p) == 1:
        return parseTree(p[0])
    if len(p) == 2:
        assert( p[0] == "not" )
        return Negation(parseTree(p[1]))
    
    if len(p) == 3:
        if p[1] == "or": return Disjunction(parseTree(p[0]), parseTree(p[2]))
        raise Exception("length 3 list but unrecognized middle token")
    if len(p) == 4:
        if p[0] == "if" and p[2] == "then":
            return Conditional(parseTree(p[1]),parseTree(p[3]))
    raise Exception("length of list unrecognized")

terms = pp.Word(pp.alphas) | "not" | "and" | "or" | "if" | "then" | "iff" 
nesting = pp.nestedExpr( '(', ')', content=terms )

def shape(s): 
    if s.strip() == "": return ""
    p = nesting.parseString("("+s+")")[0]
    
    return parseTree(p)

In [15]:
### NEW CODE ###
def disjunction_elimination(sup1,sup2,conc):
    # For disjunction elimination, one proposition must be a disjunction
    # and the other must be a negation.
    disj, neg = sup1, sup2
    if type(disj) != Disjunction: 
        disj, neg = sup2, sup1
    if type(disj) != Disjunction or type(neg) != Negation: 
        return False
  
    # In order to reach this line of the function, we must have a disjunction
    # in `disj` and negation in `neg`.  
    
    # Now in order to be an instance of *Disjunction Elimination*, the 
    # propsition under the negation must match one of the two disjuncts.
    # Whichever disjunct matches, the other disjunct should be the final 
    # proposition in the inference (the conclusion).
    under = neg.neg
    other = disj.right
    if under != disj.left:
        other = disj.left
        if under != disj.right: 
            return False
    return other == conc

# disj = shape("P or Q")
# neg = shape("not P")
# conc = shape("Q")

# print(disjunction_elimination(disj,neg,conc))

In [16]:
### BORING CODE ###
def rows_list(argument):
    level = 0
    bucket = []
    row = ""
    for c in argument:
        if c == ")": 
            level -= 1
            if level == 0:
                bucket.append(row)
                row = ""
        if level > 0: row += c
        if c == "(": level += 1
    return bucket


In [17]:

def incr_rownum(num, reason_name):
    if reason_name == "Assumption":
        num += [1]
    elif reason_name == "End Subproof":
        num = num[:-1]
        num[-1] += 1
    else:
        num[-1] += 1


In [18]:

def row_string_to_list(rs):
    bucket = rs.split(".")[:-1]
    for i in range(len(bucket)):
        bucket[i] = int(bucket[i])
    return bucket


In [19]:

def x_sees_y(x, y):
    n1, n2 = len(y), len(x)
    if n2 < n1: return False
    for i in range(n1-1):
        if y[i] != x[i]: 
            return False
    return y[n1-1] <= x[n1-1]


In [29]:

### INTERESTING CODE ###
def validate(argument):
    # Preprocess the argument into a list of lists.  
    rl = rows_list(argument)
    for i in range(len(rl)):
        rl[i] = rl[i].split("; ")
        rl[i][0] = row_string_to_list(rl[i][0])
        rl[i][1] = shape(rl[i][1])
    # When the preprocessing is done, the input 
    # "(1.; P or Q; Premise), (2.; R or (P or Q); Disjunction Introduction, 1.)" 
    # looks like 
    # [[[1], Disjunction(...), "Premise"], \
    #  [[2], Disjunction(...), "Disjunction Introduction, 1."]]

    # We will validate the row numbering of the argument, by 
    # building our own row numbering as we iterate through the 
    # list, and check that it matches the given numbering.
    rownum = [1]
    for i, row in enumerate(rl):
        
        if rownum != row[0]:
            raise Exception("Invalid row numbering.")
        
        reason = row[2].split(", ")
        reason_name = reason[0]
        incr_rownum(rownum, reason_name)

        if reason_name in ["Premise", "Assumption"]: continue

        # Validating *Disjunction Elimination* requires 
        if reason_name == "Disjunction Elimination":
            # finding the referenced row numbers
            sup_ind1 = row_string_to_list(reason[1]) 
            sup_ind2 = row_string_to_list(reason[2])

            # checking that they're not in a closed 
            # subproof
            if not (x_sees_y(row[0],sup_ind1) and \
                    x_sees_y(row[0],sup_ind2)):
                return False
            
            # getting the row at these numbers
            for r in rl:
                if r[0] == sup_ind1:
                    sup1 = r[1]
                if r[0] == sup_ind2:
                    sup2 = r[1]
            
            # and checking that they match the pattern.
            if not disjunction_elimination(sup1,sup2,row[1]):
                return False
        
        if reason_name == "Conditional Introduction":
            cond_prop = row[1]
            next_row = rl[i+1]
            if type(cond_prop) != Conditional:
                return False
            # Conditional introduction checks the next row to see
            # that it's the beginning of an assumption with the 
            # same proposition as is the antecedent of the 
            # conditional.
            cert = next_row[2] == "Assumption"
            cert *= cond_prop.ante == next_row[1]
            if not cert: return False
            # Then it finds the closing "End Subproof", and checks
            # that on the line before, the subproof ended at the 
            # consequent of the conditional.
            cert = False
            for j in range(i+1,len(rl)):
                if len(rl[j][0]) == len(next_row[0]) \
                    and rl[j][2] == "End Subproof" \
                    and rl[j-1][1] == row[1].conseq:
                    cert = True
            if not cert: return cert
        
        return True

        
# disj_argument = "(1.; P or Q; Premise), (2.; not Q; Premise), (3.; P; Disjunction Elimination, 1., 2.)"
# print(validate(disj_argument))

cond_argument = "(1.; if P then Q; Conditional Introduction), (1.1.; P; Assumption), (1.2.; Q; Premise), (1.3.; ; End Subproof))"
print(validate(cond_argument))


True


In [2]:
from manim import *
config.media_width = "100%"

Manim Community v0.17.3

In [32]:
%%manim -qm -v WARNING R2

class R2(Scene):
    def construct(self):
        plane = NumberPlane()
        self.play(Create(plane))

        vec = Arrow([0,0,0],[1,2,0], buff=0, color="RED")
        self.play(Create(vec))

        vec2 = Arrow([0,0,0],[-2,1,0], buff=0, color="RED")
        self.play(Transform(vec,vec2))
        self.remove(vec)

        vec3 = Arrow([0,0,0],[-1,-3,0], buff=0, color="RED")
        vec4 = Arrow([0,0,0],[2,0,0], buff=0, color="RED")
        vec5 = Arrow([-1,-3,0],[0,-2,0], buff=0, color="YELLOW")

        self.play(Transform(vec2,vec4))
        self.remove(vec2)
        self.play(Transform(vec4,vec3))
        self.play(Create(vec5))

        label1 = MathTex(r"\begin{bmatrix} -1\\-2\end{bmatrix}", color=RED).move_to([-1,0,0])
        self.play(Write(label1))

        label2 = MathTex(r"\begin{bmatrix} 1\\1\end{bmatrix}", color=YELLOW).move_to([1,-1,0])
        vec6 = Arrow([0,0,0],[1,1,0], buff=0)
        self.play(Write(label2))

        label3 = MathTex(r"\begin{bmatrix}1\\1\end{bmatrix}").move_to([1,2,0])
        self.play(Create(vec6))
        self.play(Write(label3))

        self.wait(3)
        

In [22]:
%%manim -qm -v WARNING R3

class R3(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=2*PI/5, theta=PI/5)
        space = ThreeDAxes()
        self.play(Create(space))

        vec = Arrow([0,0,0],[1,2,-3], buff=0, color="RED")
        vec2 = Arrow([0,0,0],[-2,1,3], buff=0, color="RED")
        vec3 = Arrow([0,0,0],[-1,-3,3], buff=0, color="RED")
        vec4 = Arrow([0,0,0],[2,0,-4], buff=0, color="RED")
        vec5 = Arrow([-1,-3,0],[0,-2,-2], buff=0, color="YELLOW")
        vec6 = Arrow([0,0,0],[1,1,0], buff=0)

        self.play(Create(vec))

        self.play(Transform(vec,vec2))
        self.remove(vec)

        self.play(Transform(vec2,vec4))
        self.remove(vec2)

        self.play(Transform(vec4,vec3))
        self.play(Create(vec5)) 

        label1 = MathTex(r"\begin{bmatrix} -1\\-3\\3\end{bmatrix}", color=RED).move_to([-2,-2,2])
        self.play(Write(label1))

        label2 = MathTex(r"\begin{bmatrix} 1\\1\\-2\end{bmatrix}", color=YELLOW).move_to([.5,-1.5,-2.5])
        self.play(Write(label2))

        label3 = MathTex(r"\begin{bmatrix}1\\1\\0\end{bmatrix}").move_to([1,2,0])
        self.play(Create(vec6))
        self.play(Write(label3))

        self.move_camera(phi=2*PI/5, theta=-85 * DEGREES, zoom=.5, run_time=1.5)
        self.move_camera(phi=1.5*PI/5, theta=-85 * DEGREES, zoom=.5, run_time=1.5)

        self.wait(3)
        